# 6장. 하이퍼파라미터 튜닝
- 방법
    - 만족할 만한 하이퍼파라미터들의 값의 조합을 찾을 때 까지 수동조정
    - GridSearchCV()
    - RandomizedSearchCV()

---
---

## 01. 일반모델

In [40]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [19]:
cancer = load_breast_cancer()
X, y = cancer.data , cancer.target

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, stratify=y, random_state=1)

In [23]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(X_train,y_train)

DecisionTreeClassifier()

In [24]:
from sklearn.metrics import accuracy_score
pred_train = dt.predict(X_train)
pred_test = dt.predict(X_test)

print('Train 정확도:', accuracy_score(y_train,pred_train))
print('Test 정확도:', accuracy_score(y_test,pred_test))

Train 정확도: 1.0
Test 정확도: 0.9415204678362573


---
---

## 02. Grid Search를  이용한 하이퍼파라미터 튜닝
- GridSearchCV()
- 시도해볼 하이퍼파라미터들을 지정하면 모든 조합에 대해 교차검증 후 제일 좋은 성능을 내는 하이퍼파라미터 조합을 찾아준다.
- 적은 수의 조합의 경우는 괜찮지만, 시도할 하이퍼파라미터와 값들이 많아지면 많은 시간이 소요된다.
- 주요 매개변수
    - estimator : 모델객체 지정 (classifier, regressor, pipeline 이 사용될 수 있음)
    - params : 하이퍼파라미터 목록은 dictionary로 전달 '파라미터명:[파라미터값 list] 형식
    - scoring : 평가지표
    - cv : 교차검증시 fold 개수
    - n_jobs : 사용할 CPU 코어 개수(None:1(default), -1:모든 코어 다 사용)
- 메소드
    - fit(X,y) : 학습
    - predict(X) : 제일 좋은 성능을 낸 모델로 predict()
    - predict_proba(X) : 제일 좋은 성능을 낸 모델로 predict_proba() 호출
- 결과 조회 변수
    - cv_results_ : 파라미터 조합별 결과 조회
    - best_params_ : 가장 좋은 성능을 낸 parameter 조합 조회
    - best_estimator_ : 가장 좋은 성능을 낸 모델 반환

In [25]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

In [50]:
dt = DecisionTreeClassifier()
param_grid = {
    'max_depth' : [4, 5, 6, 7],
    'max_leaf_nodes' : [3, 5, 7, 8, 10],
    'criterion' : ['gini','entropy'] # 기본값 말고 다른값을 사용할 경우.
}

gs = GridSearchCV(dt,
                 param_grid=param_grid,
                 scoring='accuracy',
                 cv = 3,
                 n_jobs=-1)
gs.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [4, 5, 6, 7],
                         'max_leaf_nodes': [3, 5, 7, 8, 10]},
             scoring='accuracy')

In [51]:
scores_df = pd.DataFrame(gs.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', 'split0_test_score',
           'split1_test_score', 'split2_test_score']]
## 7번째행이 best_score이다.

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'criterion': 'gini', 'max_depth': 4, 'max_lea...",0.934761,4,0.924812,0.909774,0.969697
1,"{'criterion': 'gini', 'max_depth': 4, 'max_lea...",0.937267,3,0.917293,0.924812,0.969697
2,"{'criterion': 'gini', 'max_depth': 4, 'max_lea...",0.927185,11,0.924812,0.909774,0.946970
3,"{'criterion': 'gini', 'max_depth': 4, 'max_lea...",0.924679,14,0.932331,0.894737,0.946970
4,"{'criterion': 'gini', 'max_depth': 4, 'max_lea...",0.899597,35,0.909774,0.849624,0.939394
5,"{'criterion': 'gini', 'max_depth': 5, 'max_lea...",0.934761,4,0.924812,0.909774,0.969697
6,"{'criterion': 'gini', 'max_depth': 5, 'max_lea...",0.942318,1,0.917293,0.924812,0.984848
7,"{'criterion': 'gini', 'max_depth': 5, 'max_lea...",0.924679,14,0.924812,0.902256,0.946970
8,"{'criterion': 'gini', 'max_depth': 5, 'max_lea...",0.919629,29,0.909774,0.917293,0.931818
9,"{'criterion': 'gini', 'max_depth': 5, 'max_lea...",0.907135,34,0.909774,0.864662,0.946970


In [52]:
print('GridSearchCV 최적 파라미터:',gs.best_params_)
print('GridSearchCV 최고 정확도:',gs.best_score_)
print('GridSearchCV 최고성능 모델:',gs.best_estimator_)

GridSearchCV 최적 파라미터: {'criterion': 'gini', 'max_depth': 5, 'max_leaf_nodes': 5}
GridSearchCV 최고 정확도: 0.9423179160021266
GridSearchCV 최고성능 모델: DecisionTreeClassifier(max_depth=5, max_leaf_nodes=5)


In [58]:
# 어차피 refit=True가 default값 임으로 두 방법이 같은 결과!
# 1  
pred_gs = gs.predict(X_test)
# 2
estimator = gs.best_estimator_
pred_es = estimator.predict(X_test)
# 결과
print("자동으로 학습된 모델로 predict:",accuracy_score(y_test,pred_gs))
print("모델을 조회에서 다시 predict:", accuracy_score(y_test,pred_es))

자동으로 학습된 모델로 predict: 0.9532163742690059
모델을 조회에서 다시 predict: 0.9532163742690059
